# K-Nearest Neighbour Model

## Importing libraries

In [232]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd

## Reading in CSV data
Reading the dataset from a CSV into a DataFrame for use with the model.
Remove unnecessary data and split into data and labels.

In [233]:
music_df = pd.read_csv(r'D:\Uni\Y3\FYP\Code\Music Classification\extracted-features.csv')

# Read in CSV as a Pandas DataFrame
# print(music_df)
#Drop the TrackNum from the DataFrame (This is not important for classification)
music_df.drop('TrackNum', 1, inplace = True)
# Check the field has been dropped correctly
# print(music_df)

# This will contain the data
data = music_df.iloc[:, 1:5].values
# This will contain the labels corresponding to the data
labels = music_df.iloc[:, 0].values

## Produce KNN model

In [234]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size = 0.30)

knn_model = KNeighborsClassifier()

param_grid = {"n_neighbors": range(1,25),
              "weights": ["uniform", "distance"],
              "metric": ["euclidean", "manhattan"]}

scoring_metrics = {"Accuracy": "accuracy", "AUC": "roc_auc_ovr", "F1": "f1_macro"}

# roc_auc_ovr sets the score to use roc_auc in one vs rest mode
# StratifiedKFold is used by default for the cross validation
knn_grid_search_cv = GridSearchCV(knn_model, param_grid,
                                  scoring = scoring_metrics, refit = "AUC", cv = 5)

knn_grid_search_cv.fit(data_train, labels_train)

{'metric': 'manhattan', 'n_neighbors': 23, 'weights': 'distance'}
